# This notebook is the last step in preparing the data the Random Forest Regressors for Each population to Predict Biomass

 Here we will seperate the dataframes by population, prepare split features (variables used for prediction) and the label (what we are trying to predict, biomass).

This notebook also includes a few functions we will use within the notebook and within each population's random forest model building notebooks.

#### Set a working directory

In [78]:
# Set a working directory
#!pip install GitPython
import git
import os

repo = git.Repo('.', search_parent_directories=True)


os.chdir(repo.working_tree_dir)


# Developing a Random Forest Regressor for Each PopulationName to Predict Biomass (pgC per L)

In [79]:
import pandas as pd

covari_path = 'data/modified/RF_ready_covari.csv'
#using pandas to read in as a df
covari = (pd.read_csv(covari_path,parse_dates=[0]))
#taking a peak at the data
covari.head(3)


,time,population,lat,lon,biomass,salin,temp,cruisename,SiO2,POC,...,O2,NO3,NO2,NH4,FeT,DOC,DIC,CDOM,ALK,par
0,2016-04-20 07:00:00,Prochlorococcus,21.520326,-158.326984,10.520443,34.893785,24.351745,KOK1606,-0.022845,0.032884,...,214.191393,2.252042,0.295276,1.282981,0.000015,1.648093,1697.874775,0.000034,1954.87665,0.0193
1,2016-04-20 07:00:00,Synechococcus,21.520326,-158.326984,0.341429,34.893785,24.351745,KOK1606,-0.022845,0.032884,...,214.191393,2.252042,0.295276,1.282981,0.000015,1.648093,1697.874775,0.000034,1954.87665,0.0193
2,2016-04-20 07:00:00,nanoeukaryotes (2-5µm),21.520326,-158.326984,3.338212,34.893785,24.351745,KOK1606,-0.022845,0.032884,...,214.191393,2.252042,0.295276,1.282981,0.000015,1.648093,1697.874775,0.000034,1954.87665,0.0193


## Seperating dataframes for individual PopulationNames
We have four populations present in the dataset.

In [80]:
covari.population.unique()

array(['Prochlorococcus', 'Synechococcus', 'nanoeukaryotes (2-5µm)',
       'picoeukaryotes (< 2µm)'], dtype=object)

In [81]:
import numpy as np

def preprocess_single_population(covari, population_name):
    """
    Takes the covari dataframe, and whichever of the four populations and returns a dataframe that
    only includes the selected population's rows, a list of labels (biomass values associated with the dataframe)
    and a list of all of the features.
    """
    # Selecting the population based on the provided name
    pop_df = covari[covari.population == population_name]

    # Creating the labels and features for the population
    labels = np.array(pop_df.biomass)
    features = pop_df.drop(['time', 'population', 'biomass', 'lat', 'lon', 'cruisename'], axis=1)
    # Saving feature names for later use
    feature_list = list(features.columns)
    features = features.to_numpy()
    if features.shape[0] > 1:  # Check if features has more than one row
        features = np.delete(features, 0, 0)
    return pop_df, labels, features, feature_list




### Using the preprocess_single_population function for all of the populations

In [82]:
# creating a df, labels and features with only for the observed Prochlorooccus
population_name = 'Prochlorococcus'

pro_df, labels_pro, features_pro, feature_list_pro = preprocess_single_population(covari, population_name=population_name)
print(features_pro.shape)
print(type(features_pro))

(2392, 16)
<class 'numpy.ndarray'>


In [83]:
# creating a df, labels and features with only for the observed Synechoccoccus
population_name = 'Synechococcus'

syn_df, labels_syn, features_syn, feature_list_syn = preprocess_single_population(covari, population_name)

In [84]:
# creating a df, labels and features with only for the observed Picoeukaryotes
population_name = 'picoeukaryotes (< 2µm)'

pico_df, labels_pico, features_pico, feature_list_pico = preprocess_single_population(covari, population_name)

In [85]:
# creating a df, labels and features with only for the observed Nanoeukaryotes
population_name = 'nanoeukaryotes (2-5µm)'

nano_df, labels_nano, features_nano, feature_list_nano = preprocess_single_population(covari, population_name)

## Defining a function for finding the optimal testing to training ratio

Used in specific random forest model notebooks. This function graphs the Root Mean Square Error (RMSE) vs. the testing to training ratio for data used in the model. The aim is to choose the highest testing to training ratio where the RMSE starts to fall most dramatically in value. Keeping a higher testing to training ratio keeps our model more  generalizable and prevents overfitting.  This function is called in each population specific notebook to find the optimal testing to training ratio for that population's model.

In [86]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np

def testing_training_ratio(features, labels, feature_list, title_prefix):
    # Graphs the RMSE of different testing and training ratios
    RMSEs = {'Test_Ratio':[], 'RMSE': []}

    n_splits = 4
    kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)

    for train_index, test_index in kf.split(features):
        train_features, test_features = features[train_index], features[test_index]
        train_labels, test_labels = labels[train_index], labels[test_index]

        rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
        rf.fit(train_features, train_labels)

        # Use the forest's predict method on the test data
        predictions = rf.predict(test_features)

        # Calculate the absolute errors
        errors = abs(predictions - test_labels)

        # Finding the root mean square error (RMSE)
        RMSE = mean_squared_error(test_labels, predictions, squared=False) #setting squared=False gives us RMSE not MSE
        RMSEs['RMSE'].append(RMSE)
        RMSEs['Test_Ratio'].append(1/n_splits)  # The test ratio for n-fold cross-validation is 1/n_splits

    # Rest of the code...
   
    # Extract Test Ratios and RMSEs from the dictionary
    test_ratios = RMSEs['Test_Ratio']
    rmse_values = RMSEs['RMSE']

    # Create a line plot
    plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
    plt.plot(test_ratios, rmse_values, marker='o')

    # Fill the area under the curve
    plt.fill_between(test_ratios, rmse_values, alpha=0.3)
    
    plt.xlabel('Testing:Training Ratio', fontsize=15)
    plt.ylabel('RMSE of Biomass (pgC/L)fn', fontsize=15)
    plt.title(f"{title_prefix} - RMSE of Biomass vs. Testing: Training Ratio", fontsize=22)

    plt.xlim(0, 1)  # Set the x-axis limits from 0 to 1
    
    plt.xticks([i/10 for i in range(11)])  # Set the x-axis tick locations at 0.1 increments
    #inversing the x axis
    plt.gca().invert_xaxis()

    
    plt.grid(True)  # Add a grid to the plot
    
    plt.tight_layout()  # Improves spacing between the plot elements
    plt.show()
    
    return RMSEs
n_splits = 4
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)
print(features_pro)
print(labels_pro)
for train_index, test_index in kf.split(features_pro):
    print(train_index, test_index)


[[ 3.49023765e+01  2.43392647e+01 -2.28452622e-02 ...  3.37754147e-05
   1.95487665e+03  2.03823530e-02]
 [ 3.48805900e+01  2.43207250e+01 -2.19817645e-02 ...  3.09030475e-05
   1.95626175e+03  8.56500000e-02]
 [ 3.48840526e+01  2.43108263e+01 -2.19817645e-02 ...  3.09030475e-05
   1.95626175e+03  8.59157890e-02]
 ...
 [ 3.34713633e+01  1.53143378e+01  3.63295805e-01 ...  7.55719400e-04
   2.00841777e+03  5.68868550e+02]
 [ 3.34681505e+01  1.51890208e+01  3.63295805e-01 ...  7.55719400e-04
   2.00841777e+03  1.27692150e+02]
 [ 3.34788456e+01  1.53273022e+01  3.63295805e-01 ...  7.55719400e-04
   2.00841777e+03  1.01280000e+00]]
[10.52044346  9.3093869   9.4065122  ...  0.72024412  0.76845579
  0.87459865]
[   0    1    2 ... 2389 2390 2391] [  13   18   23   25   29   43   44   48   49   51   56   65   67   69
   70   73   76   84   96   99  100  109  111  118  120  124  128  134
  135  149  157  163  168  173  174  177  178  179  184  185  188  192
  196  199  203  208  210  211  212 

## Defining a function to plot out-of-bag error (OOB) againts number of trees in random forest model

This function tests different numbers of trees used in the random forest model and finds the OOB for each number of trees.  This is then called in each population-specific notebook to find the optimal number of trees for that population's model.  

In [87]:
from collections import OrderedDict
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

def plot_oob_error_vs_num_trees(train_features, train_labels, title_prefix):
    """
    Developes a plot of Out of Bag (oob) error vs the number of trees grown in a random forest model. There are
        three labeled lines within the plot one representing.
    
    Max features represent the amount of all features (varaibles we are predicting on) used for each 
        tree in the random forest. n = all features.
    
    Warm start = true:reuse the solution of the previous call to fit and add more
        estimators to the ensemble, otherwise, just fit a whole new forest.
        
    oob_score = True: Use out-of-bag samples to estimate the generalization score. By default, r2_score is used.
        Provide a callable with signature.
    
    random state: controls random number generator that is used to shuffle/split the data. Ensures the same
        randomization is used each time the code is ran.
    
    """
    RANDOM_STATE = 42

    ensemble_clfs = [
        (
            "max_features='sqrt(n)'",
            RandomForestRegressor(
                warm_start=True,
                max_features="sqrt",
                oob_score=True,
                random_state=RANDOM_STATE,
            ),
        ),
        (
            "max_features='1/3 n'",
            RandomForestRegressor(
                warm_start=True,
                max_features=1/3,
                oob_score=True,
                random_state=RANDOM_STATE,
            ),
        ),
        (
            "max_features= n",
            RandomForestRegressor(
                warm_start=True,
                max_features=None,
                oob_score=True,
                random_state=RANDOM_STATE,
            ),
        ),
    ]

    error_rate = OrderedDict((label, []) for label, _ in ensemble_clfs)

    min_estimators = 15
    max_estimators = 128

    for label, clf in ensemble_clfs:
        for i in range(min_estimators, max_estimators + 1, 5):
            oob_errors = []
            for fold_features, fold_labels in zip(train_features, train_labels):
                clf.set_params(n_estimators=i)
                clf.fit(fold_features, fold_labels)
                oob_error = 1 - clf.oob_score_
                oob_errors.append(oob_error)
            avg_oob_error = np.mean(oob_errors)
            error_rate[label].append((i, avg_oob_error))
    for label, clf_err in error_rate.items():
        xs, ys = zip(*clf_err)
        plt.plot(xs, ys, label=label)

    plt.xlim(min_estimators, max_estimators)
    plt.xlabel("# of Trees")
    plt.ylabel("OOB error rate (1 - R^2)")
    plt.legend(loc="upper right")
    plt.suptitle(f"{title_prefix} - Out-of-Bag Error Rate vs. Number of Trees in Random Forest Regression")
    plt.show()


## Defining functions to compare predicted biomass with actual data

These functions give us a preliminary look at how well the model is predicting biomass.  These are called in each of the population specific notebooks.  

In [88]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_error
import seaborn as sns

def plot_model_predictions():
    """
    Creates two plots. First a Line density plot with the Root Mean Square Percentage Error (RMSPE) displayed 
    with two line density placments one for predicted and one for actual values. The second plot displayes a 
    scatter density plot of predicted vs true values with a linear regression line.
    """
    
    import matplotlib.pyplot as plt
    import numpy as np
    from sklearn.metrics import mean_absolute_error
    import seaborn as sns

    fig, axes = plt.subplots(1, 2, figsize=(14, 7))
    
    # First subplot: Line Density Plot
    ax1 = axes[0]
    mae = mean_absolute_error(test_labels[1], predictions)
    sns.histplot(x=test_labels[1], fill=True, color="blue", label="True Values", ax=ax1)
    sns.histplot(x=predictions, fill=True, color="red", label="Predictions", ax=ax1)
    ax1.text(0.05, 0.9, 'RMSPE = {:.2f}%'.format(percentage_RMSE), transform=ax1.transAxes, fontsize=20)
    ax1.set_xlabel('Biomass (pgC per L)')
    ax1.set_ylabel('Density')
    ax1.grid(True)
    ax1.legend()

    # Second subplot: Scatter Plot with Linear Regression Line
    ax2 = axes[1]
    ax2.set_xlabel('Actual Biomass (pgC per L)')
    ax2.set_ylabel('Predicted Biomass (pgC per L)')
    ax2.set_ylim(0, max(test_labels.max(), predictions.max())-5)
    ax2.set_xlim(0, max(test_labels.max(), predictions.max())-5)
    ax2.set_aspect('equal')
    ax2.grid(True)

    
    # PRO Model Predictions vs True Values
    sns.kdeplot(x=test_labels[1], y=predictions, shade=True, cmap="Reds", ax=ax2)
    ax2.set_aspect('equal')
    ax2.grid(True)

    # Add linear regression line
    x = test_labels[1]
    y = predictions
    slope, intercept = np.polyfit(x, y, 1)
    ax2.plot(x, slope*x + intercept, color='blue', label='Linear Regression Line')
    ax2.legend(loc='upper left')  # Add legend to the upper left corner of the plot


    # Adjust spacing between subplots
    plt.tight_layout()

    # Display the plot
    
    return fig, axes



In [90]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_error
import seaborn as sns

def plot_model_predictions_density():
    """
    Displayes a  scatter density plot of predicted vs true values with a linear regression line in addition
    to a one-to-one line that displays a proportionate relationship
    """
    import matplotlib.pyplot as plt
    import numpy as np
    from sklearn.metrics import mean_absolute_error
    import seaborn as sns

    fig, ax = plt.subplots(1, 1, figsize=(7, 7))


    # Scatter Plot with Linear Regression Line
    ax.set_xlabel('Actual Biomass (pgC per L)', fontsize=24)
    ax.set_ylabel('Predicted Biomass (pgC per L)', fontsize=24)
    ax.set_ylim(0, max(test_labels.max(), predictions.max())-5)
    ax.set_xlim(0, max(test_labels.max(), predictions.max())-5)
    ax.set_aspect('equal')
    ax.grid(True)
    ax.text(0.05, 0.9, 'RMSPE = {:.2f} %'.format(percentage_RMSE), transform=ax.transAxes, fontsize=20)
    
    
    # PRO Model Predictions vs True Values
    sns.kdeplot(x=test_labels[1], y=predictions, shade=True, cmap="Reds", ax=ax)
    ax.set_aspect('equal')
    ax.grid(True)

    # Add linear regression line
    x = test_labels[1]
    y = predictions
    slope, intercept = np.polyfit(x, y, 1)
    ax.plot(x, slope*x + intercept, color='blue', label='Linear Regression')
    
    # Add black dashed one-to-one line
    max_val = max(test_labels.max(), predictions.max())
    ax.plot([0, max_val], [0, max_val], linestyle='--', color='black', label='One-to-One')


    # Adjust spacing between subplots
    plt.tight_layout()

    # Display the plot
    
    return fig, ax

